In [2]:
# import Pkg; Pkg.add("POMDPPolicies")
# Pkg.add("POMDPModelTools")
# Pkg.add("Distributions")
# Pkg.add("Combinatorics")
# Pkg.add("StaticArrays")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("TickTock")
# Pkg.add("POMDPTools")
# Pkg.add("D3Trees")
# Pkg.add("Random")
# Pkg.add("DataFrames")
# Pkg.add("CSV")

In [3]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [4]:
Number_level = 4;


gamma=0.95
Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]
# Observation_matrix=[
#     0.65  0.25 0.1   0.0;
#     0.175 0.65 0.175 0.0;
#     0.1   0.25 0.65  0.0;
#     0.0   0.0  0.0   1.0
# ]

States=Array((1:Number_level))
Observations=Array((1:Number_level))

Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstate = SparseCat(States, Initialstates)

# Initialobss = zeros(Number_level)
# Initialobss[1]=1.0
# Initialobs = SparseCat(States, Initialobss)
Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [5]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

## Observation function

In [6]:
function observation_function(s)
    return SparseCat(Observations, Observation_matrix[s,:])
end
# obs_std=[1,1.2,1.5]
# obs_mean=[0.5,1,1.5]

# function observation_function(s)
#     if s==Number_level
#         return Deterministic(Number_level)
#     else
#         cont_dist=Normal(obs_mean[s],obs_std[s])
#         probs=zeros(Number_level)
#         for i in 1:(Number_level-1)
#             if i==1
#                 probs[i]=cdf(cont_dist,0.75)
#             elseif i==(Number_level-1)
#                 probs[i]=1-cdf(cont_dist,1.25)
#             else
#                 probs[i]=cdf(cont_dist,i+0.25)-cdf(cont_dist,i-0.25)
#             end
#         end
#         probs ./= sum(probs)
#         return SparseCat(1:(Number_level),probs)
#     end
# end


observation_function (generic function with 1 method)

## Belief updater

In [7]:
function update_belief(b,s_b,a,o) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    updated_belief = zeros(Number_level)
    if o==Number_level
        updated_belief[Number_level]=1.0
        return updated_belief
    end
    
    if s_b >= a
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: Number_level
            #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
            p_obs=pdf(observation_function(sp),o)
            p_trans = pdf(SparseCat(States,Transition_matrix[1,:]),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:Number_level
        #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
        p_obs=pdf(observation_function(sp),o)
        p_trans = 0.0
        for s in 1:Number_level
            p_trans += pdf(SparseCat(States,Transition_matrix[s,:]),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end

update_belief (generic function with 1 method)

In [8]:
NumberUnits = 10
belief = [[0.8, 0.2, 0.0, 0.0] for j in 1:NumberUnits]
b = belief[1,:]

1-element Vector{Vector{Float64}}:
 [0.8, 0.2, 0.0, 0.0]

In [9]:
b = belief[1]

4-element Vector{Float64}:
 0.8
 0.2
 0.0
 0.0

In [10]:
p_trans = 0.0
s = 1
sp = 2

p_trans += pdf(SparseCat(States,Transition_matrix[s,:]),sp) * b[s]

0.11432

## Define POMDP

In [11]:
# s is the system-level state vector
#s,r,o,true_a=approximate(s,s_belief,local_a,234) 
# Approximating the solution by taking the state of highest belief as the true state
function approximate(s,s_belief,a,rng=234)
    r=0.0
    prevent_repair= false
    failed_components=count(x->x==Number_level,s_belief)
    if failed_components >= limit
        r += system_penalty
        # if a==Number_level +1
        #     a = Number_level
        # end
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a 
            # Maintenance action is decided based on belief, not true state
            # since we don't know true states when making decision
            if s_belief[i] == Number_level
                r += failure_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            else
                r += maintenance_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            end
            s[i]=rand(crd[1])
            r += normal_operation
        else
            s[i] = rand(crd[s[i]])
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs,a=a)
end

approximate (generic function with 2 methods)

In [12]:
# # Generate random positive numbers and normalize each row
# function generate_prob_matrix(rows, cols)
#     # Create random positive numbers
#     matrix = rand(rows, cols)
    
#     # Normalize each row to sum to 1
#     normalized = matrix ./ sum(matrix, dims=2)
    
#     return normalized
# end

# # Generate 20x4 matrix
# prob_matrix = generate_prob_matrix(20, 4)

In [13]:
# global initialb= zeros(20, 4)  
# global initialb[:, 1] .= 1.0
# s_belief = [argmax(row) for row in eachrow(initialb)]

In [14]:
# belief[1,:]=[0.3,0.3,0.4,0]
# true_a=3
# o[1]=2

In [15]:
# update_belief([0.3,0.3,0.4,0], 3,3, 1)

In [16]:
function findnN()
    println("Finding best nN policy...")
    rewards_nN = zeros(Number_level,Number_level);
    rewards_nN = rewards_nN.+(-100000000.0);
    temp_rewards = zeros(repetition,1);
    # global rewards_nN_std = zeros(Number_level,Number_level);
    # global rewards_nN_std = rewards_nN_std.+(-100000000.0);
    for N in 2:(Number_level)
        for n in 1:N
            # temp_rewards = zeros(repetition,1)
            Threads.@threads  for j in 1:repetition
            #for j in 1:repetition
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
                for k in 1:simsteps
                    # s_belief = [argmax(row) for row in eachrow(belief)]
                    s_belief = argmax.(belief)
                    local_a = Number_level +1; # when local_a = 0, no repair unless system fails
                    for i in 1:NumberUnits
                    # if there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n.
                        
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    true_a=local_a
                    # s is the system-level state vector
                    s,r,o,true_a=approximate(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        belief[i]=update_belief(belief[i], s_belief[i],true_a, o[i])
                    end
                    append!(R_sequence,r)
                end
                R_discounted=0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                temp_rewards[j]=R_discounted
            end
            rewards_nN[n,N] = mean(temp_rewards)
            #rewards_nN_std[n,N] = std(temp_rewards)
        end
    end
    # This max_reward is actually wrong in POMDP
    # The (n, N) obtained should be calculated based on true state
    (max_reward,nN)=findmax(rewards_nN)
    
    #println("Max rewards of (n, N)-policy is", max_reward, "n is ", nN[1], "N is ", nN[2])
    #return max_reward, rewards_nN_std[nN], nN
    return nN
end

findnN (generic function with 1 method)

In [17]:
# function generative(s,s_belief,a,rng=234)
#     r=0.0
#     prevent_repair= false
#     failed_components=count(x->x==Number_level,s_belief)
#     if failed_components >= limit
#         r += system_penalty
#         if a==Number_level +1
#             a = Number_level
#         end
#     end

#     for i in 1:NumberUnits
#         if s_belief[i] >= a
#             if s[i] == Number_level
#                 r += failure_penalty
#                 if prevent_repair == false
#                     r += setup_cost
#                     prevent_repair = true
#                 end
#             else
#                 r += maintenance_penalty
#                 if prevent_repair == false
#                     r += setup_cost
#                     prevent_repair = true
#                 end
#             end
#             s[i]=rand(crd[1])
#             r += normal_operation
#         else
#             s[i] = rand(crd[s[i]])
#             r = r+normal_operation
#         end
#     end

#     obs=repeat(1:1,NumberUnits)
#     for i in 1:NumberUnits
#         #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
#         obs_distribution = observation_function(s[i])
#         obs[i] = rand(obs_distribution)
#     end

#     return (s=s,r=r,o=obs,a=a)
# end

In [18]:
function generative(s,s_belief,a,rng=234)
    r=0.0
    # prevent_repair= false
    # failed_components=count(x->x==Number_level,s_belief)
    # if failed_components >= limit
    #     #r += system_penalty
    #     if a==Number_level +1
    #         a = Number_level
    #     end
    # end

    if a <= Number_level
         r += setup_cost
    end

    true_failed_components=count(x->x==Number_level,s)
    if true_failed_components >= limit
        r += system_penalty
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a
            if s[i] == Number_level
                r += failure_penalty
            else
                r += maintenance_penalty
            end
            s[i]=rand(crd[1])
            r += normal_operation
        else
            s[i] = rand(crd[s[i]])
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        #obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs)
end

generative (generic function with 2 methods)

In [19]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],mean=[],std=[],nN=[])

cost=[ 
    [0	-800	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 


for units in [20, 25, 30, 40, 50, 60]

    for K in 2:2:convert(Int64, floor((units+1)/2)) 
        
        global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)
        global crd = Array{Categorical}(undef,Number_level)
        for i in 1:Number_level
            global crd[i] = Categorical(Transition_matrix[i,:]);
        end
            
        # global initialb= zeros(NumberUnits, Number_level)  
        # global initialb[:, 1] .= 1.0
        for c in cost
            tick()
            println(units)  
            print("K : ")
            println(K)
            global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
            println(c)  
            global NumberUnits=units

            global simsteps = 100;
            global repetition = 10000;
            global rewards_nN = zeros(Number_level,Number_level);
            global rewards_nN = rewards_nN.+(-100000000.0);
            global rewards_nN_std = zeros(Number_level,Number_level);
            global rewards_nN_std = rewards_nN_std.+(-100000000.0);

            global discount_factor =gamma
            global temp_rewards = zeros(repetition,1);
            # Looking for optimal (n, N)
            nN =findnN()
            n=nN[1]
            N=nN[2]
            
            trials=10000;
            results=zeros(trials)*0.1
            # cnt_maintenance = zeros(trials)
            rewards=0.0
            rewards_std=0.0
            Threads.@threads for h in 1:trials
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
              
                
                # maintenance_count = 0
                for k in 1:simsteps
                    # s_belief = [argmax(row) for row in eachrow(belief)]
                    s_belief = argmax.(belief)
                    # if h==1
                    #     println("The belief vector is ", belief)
                    #     println("====================================================")
                    #     println("The maximum likelihood state vector is", s_belief)
                    # end
                    local_a = Number_level +1; 
                    for i in 1:NumberUnits
                    # If there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n,
                    # otherwise, no maintenance.
                    # As a result, when system fails, there are components at failure level,
                    # so N must be reached.
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    # true_a=local_a
                    # s is the system-level state vector
                    # if local_a <= Number_level
                    #     # println("maintain")
                    #     maintenance_count += 1
                    # end
                    s,r,o=generative(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        # modified: belief[i,:] to belief[i]
                        belief[i]=update_belief(belief[i], s_belief[i],local_a, o[i])

                    end
                    append!(R_sequence,r)
                end
                R_discounted=0.0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                # cnt_maintenance[h]=maintenance_count
                results[h]=R_discounted
            end
            rewards=mean(results)
            rewards_std=std(results)
            # N_maintenance = mean(cnt_maintenance)
            println("Rewards of (n, N)-policy is", rewards, "n is ", nN[1], "N is ", nN[2])
            # println("Number of maintenance times is ", N_maintenance)
            pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,rewards,rewards_std,nN])
        end
    end
end

20
K : 2


[ Info:  started timer at: 2025-03-05T16:20:55.970


[0 -800 -100 -1200 -2000]
Finding best nN policy...
Rewards of (n, N)-policy is-11391.02628593657n is 2N is 4
20
K : 2
[0 -1200 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-05T16:31:43.526


Rewards of (n, N)-policy is-12640.356296794056n is 2N is 4
20
K : 2
[0 -1600 -100 -1200 -2000]
Finding best nN policy...


...

Rewards of (n, N)-policy is-30200.146179459727n is 2N is 4
60
K : 30
[0 -2800 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-12T05:16:12.103


Rewards of (n, N)-policy is-32030.43886929825n is 2N is 4
60
K : 30
[0 -3200 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-12T05:46:47.625


Rewards of (n, N)-policy is-33853.66417469103n is 2N is 4


In [20]:
df

Row,u,K,n,s,m,f,p,mean,std,nN
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,60,30,0,-3200,-100,-1200,-2000,-33853.7,2696.62,"CartesianIndex(2, 4)"
2,60,30,0,-2800,-100,-1200,-2000,-32030.4,2557.38,"CartesianIndex(2, 4)"
3,60,30,0,-2400,-100,-1200,-2000,-30200.1,2346.64,"CartesianIndex(2, 4)"
4,60,30,0,-2000,-100,-1200,-2000,-28332.2,2197.88,"CartesianIndex(2, 4)"
5,60,30,0,-1600,-100,-1200,-2000,-26525.4,2021.93,"CartesianIndex(2, 4)"
6,60,30,0,-1200,-100,-1200,-2000,-24724.3,1883.4,"CartesianIndex(2, 4)"
7,60,30,0,-800,-100,-1200,-2000,-22884.2,1722.09,"CartesianIndex(2, 4)"
8,60,28,0,-3200,-100,-1200,-2000,-33823.6,2712.86,"CartesianIndex(2, 4)"
9,60,28,0,-2800,-100,-1200,-2000,-32062.1,2510.06,"CartesianIndex(2, 4)"


In [21]:
using CSV
CSV.write("Homo (n,N).csv", df)

"Homo (n,N).csv"